In [1]:
#!conda install -y gdown
#!gdown --id 1feedmX0pDTltwE1uwmgIzIP7KFeNRkU9
#!unzip '/content/gdrive/MyDrive/22_23_sem1/col870/*.zip'
#!for z in '/kaggle/working/*.zip'; do unzip -q "$z"; done

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    filelock-3.8.2             |     pyhd8ed1ab_0          14 KB  conda-forge
    gdown-4.6.0                |     pyhd8ed1ab_0          18 KB  conda-forge
    openssl-1.1.1s             |       h0b41bf4_1         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages

In [4]:
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random


In [2]:
train_data_dir = '../input/data-images/4/train' # put path of training dataset
val_data_dir = '../input/data-images/4/val' # put path of test dataset
test_data_dir = '../input/data-images/4/test' # put path of test dataset

In [5]:

########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].

# Apply necessary image transfromations here 

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])

trainset = torchvision.datasets.ImageFolder(root= train_data_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

valnset = torchvision.datasets.ImageFolder(root= val_data_dir, transform=transform)
valloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root= test_data_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


In [20]:
########################################################################
# Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        
        self.norm1 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc3 = nn.Linear(in_features=256, out_features=33) # 5 is the number of classes here (for batch 3,4,5 out_features is 33)
 
    def forward(self, x): 
 
        x = F.relu(self.conv1(x))
        #x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        #x = self.pool(x)
        
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        
        return x 


################### DO NOT EDIT THE BELOW CODE!!! #######################

net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')

785121  parameters


In [21]:
########################################################################
# Train the network
# ^^^^^^^^^^^^^^^^^^^^

def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))
    

In [22]:
train(epoch, trainloader, optimizer, criterion)

 13%|█▎        | 417/3300 [01:33<10:45,  4.46it/s]


KeyboardInterrupt: 

In [9]:
########################################################################
# Let us look at how the network performs on the test dataset.
l=[]
def test(testloader, model,s):
    global test_num
    test_num+=1
    correct = 0
    total = 0
    l=[]
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if test_num%2==1:
      print('Accuracy of the network on the %s images: %f %%' % (s,
                                    100 * correct / total))
    else:
        print('Accuracy of the network on the %s images: %f %%' % (s,
                                    100 * correct / total))
        l.append(100 * correct / total)

#########################################################################
# get details of classes and class to index mapping in a directory
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    classes, _ = find_classes(train_data_dir)
    n_class = len(classes) # number of classes

    class_correct = list(0. for i in range(n_class))
    class_total = list(0. for i in range(n_class))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(n_class):
        print('Accuracy of %10s : %2f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [10]:
#test base without quantization
checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
net.load_state_dict(checkpoint)
test(trainloader,net,"training")
test(testloader, net,"test")
classwise_test(testloader, net)

100%|██████████| 3300/3300 [04:35<00:00, 11.98it/s]


Accuracy of the network on the training images: 92.742424 %


100%|██████████| 825/825 [01:08<00:00, 12.12it/s]


Accuracy of the network on the test images: 55.181818 %


100%|██████████| 825/825 [01:10<00:00, 11.74it/s]

Accuracy of Ibizan_hound : 20.000000 %
Accuracy of aircraft_carrier : 71.000000 %
Accuracy of beer_bottle : 46.000000 %
Accuracy of     bolete : 63.000000 %
Accuracy of      boxer : 55.000000 %
Accuracy of     carton : 32.000000 %
Accuracy of       dome : 57.000000 %
Accuracy of electric_guitar : 22.000000 %
Accuracy of       file : 63.000000 %
Accuracy of french_bulldog : 37.000000 %
Accuracy of garbage_truck : 55.000000 %
Accuracy of golden_retriever : 72.000000 %
Accuracy of gordon_setter : 64.000000 %
Accuracy of hair_slide : 59.000000 %
Accuracy of  hourglass : 44.000000 %
Accuracy of house_finch : 69.000000 %
Accuracy of   komondor : 64.000000 %
Accuracy of   malamute : 62.000000 %
Accuracy of    meerkat : 65.000000 %
Accuracy of pencil_box : 50.000000 %
Accuracy of prayer_rug : 38.000000 %
Accuracy of       reel : 31.000000 %
Accuracy of rock_beauty : 78.000000 %
Accuracy of scoreboard : 67.000000 %
Accuracy of solar_dish : 46.000000 %
Accuracy of      stage : 63.000000 %
Accura

In [17]:
########################################################################
# Quantizing the Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.quantization as qt

num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
         #QuantStub converts tensors from floating point to quantized
        self.quant = qt.QuantStub()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.relu1 = nn.ReLU(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)
        self.relu3 = nn.ReLU(inplace=True)
        self.relu4 = nn.ReLU(inplace=True)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc3 = nn.Linear(in_features=256, out_features=33) # 5 is the number of classes here (for batch 3,4,5 out_features is 33)
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = qt.DeQuantStub()
 
    def forward(self, x): 
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.relu1(self.conv1(x))
        #x = self.pool(x)
        
        x = self.relu2(self.conv2(x))
        x = self.pool(x)
        
        x = self.relu3(self.conv3(x))
        x = self.pool(x)
        
        x = self.relu4(self.conv4(x))
        #x = self.pool(x)
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x 


################### DO NOT EDIT THE BELOW CODE!!! #######################

net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')


785121  parameters


In [24]:
#test dynamic quantization
checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
net.load_state_dict(checkpoint)
model_net8 = torch.quantization.quantize_dynamic(net, {torch.nn.Linear}, dtype=torch.qint8)
model_net8.to(torch.device('cpu'))
model_net8.eval()
test(trainloader,model_net8,"train")
test(testloader, model_net8,"test")
classwise_test(testloader, model_net8)
os.makedirs('./models', exist_ok=True)
torch.save(model_net8.state_dict(), './models/bestmodel_d.pth')

100%|██████████| 3300/3300 [04:05<00:00, 13.43it/s]


Accuracy of the network on the train images: 92.651515 %


100%|██████████| 825/825 [00:59<00:00, 13.77it/s]


Accuracy of the network on the test images: 55.212121 %


100%|██████████| 825/825 [00:59<00:00, 13.85it/s]

Accuracy of Ibizan_hound : 20.000000 %
Accuracy of aircraft_carrier : 72.000000 %
Accuracy of beer_bottle : 47.000000 %
Accuracy of     bolete : 64.000000 %
Accuracy of      boxer : 56.000000 %
Accuracy of     carton : 32.000000 %
Accuracy of       dome : 58.000000 %
Accuracy of electric_guitar : 22.000000 %
Accuracy of       file : 62.000000 %
Accuracy of french_bulldog : 39.000000 %
Accuracy of garbage_truck : 53.000000 %
Accuracy of golden_retriever : 73.000000 %
Accuracy of gordon_setter : 65.000000 %
Accuracy of hair_slide : 59.000000 %
Accuracy of  hourglass : 43.000000 %
Accuracy of house_finch : 69.000000 %
Accuracy of   komondor : 64.000000 %
Accuracy of   malamute : 62.000000 %
Accuracy of    meerkat : 65.000000 %
Accuracy of pencil_box : 49.000000 %
Accuracy of prayer_rug : 37.000000 %
Accuracy of       reel : 31.000000 %
Accuracy of rock_beauty : 78.000000 %
Accuracy of scoreboard : 66.000000 %
Accuracy of solar_dish : 46.000000 %
Accuracy of      stage : 63.000000 %
Accura

In [23]:
#test static quantization

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.quantization as qt

num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
         #QuantStub converts tensors from floating point to quantized
        self.quant = qt.QuantStub()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
       
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)
        self.relu3 = nn.ReLU(inplace=False)
        self.relu4 = nn.ReLU(inplace=False)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc3 = nn.Linear(in_features=256, out_features=33) # 5 is the number of classes here (for batch 3,4,5 out_features is 33)
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = qt.DeQuantStub()
 
    def forward(self, x): 
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.relu1(self.conv1(x))
        #x = self.pool(x)
        x = self.relu4(self.conv4(x))
        
        x = self.pool(x)
        
        x = self.relu2(self.conv2(x))
        
        #x = self.pool(x)
        x = self.relu3(self.conv3(x))
        
        x = self.pool(x)
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x 


################### DO NOT EDIT THE BELOW CODE!!! #######################

net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')

model_fp32 = net
model_fp32.eval()
checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
model_fp32.load_state_dict(checkpoint)
model_fp32.eval()
print(model_fp32.conv1)
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32, [['conv1', 'relu1'],['conv4', 'relu4'],['conv2', 'relu2'],['conv3', 'relu3'],]
)
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

# quantization algorithm calibration happens here
print('Start Calibration')
os.makedirs('./models', exist_ok=True)


for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train_quant(epoch, trainloader, optimizer, criterion,model_fp32_prepared)
    #test(valloader, model_fp32_prepared)
    #classwise_test(valloader, net)
    test(testloader, model_fp32_prepared)
    # save model checkpoint 
    torch.save(model_fp32_prepared.state_dict(), './models/model'+str(epoch)+'.pth')      

print('performing test')
#test(testloader, net)
classwise_test(testloader, net)
print("max accuracy:",np.max(l))
print('Finished Calibration')

model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=True)
print(model_int8.conv1)

test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)
os.makedirs('./models', exist_ok=True)
torch.save(model_net8.state_dict(), './models/bestmodel_s.pth')

785121  parameters
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Start Calibration
epoch  1


NameError: name 'train_quant' is not defined

In [ ]:
from torch.quantization.observer import MinMaxObserver, MovingAverageMinMaxObserver, HistogramObserver
torch.save(model_fp32_prepared.state_dict(), './models/model_sc.pth')
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=True)
print(model_int8.conv1)

test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)
os.makedirs('./models', exist_ok=True)
torch.save(model_net8.state_dict(), './models/bestmodel_sc1.pth')

In [13]:
#test static quantization

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.quantization as qt
from torch.quantization.observer import MinMaxObserver, MovingAverageMinMaxObserver, HistogramObserver


num_epochs = 20        # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
         #QuantStub converts tensors from floating point to quantized
        self.quant = qt.QuantStub()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
       
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=False)
        self.relu2 = nn.ReLU(inplace=False)
        self.relu3 = nn.ReLU(inplace=False)
        self.relu4 = nn.ReLU(inplace=False)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc3 = nn.Linear(in_features=256, out_features=33) # 5 is the number of classes here (for batch 3,4,5 out_features is 33)
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = qt.DeQuantStub()
 
    def forward(self, x): 
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.relu1(self.conv1(x))
        #x = self.pool(x)
        x = self.relu4(self.conv4(x))
        
        x = self.pool(x)
        
        x = self.relu2(self.conv2(x))
        
        #x = self.pool(x)
        x = self.relu3(self.conv3(x))
        
        x = self.pool(x)
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x 


################### DO NOT EDIT THE BELOW CODE!!! #######################

net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')

model_fp32 = net
checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
model_fp32.load_state_dict(checkpoint)
model_fp32.train()
print(model_fp32.conv1)
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32, [['conv1', 'relu1'],['conv4', 'relu4'],['conv2', 'relu2'],['conv3', 'relu3'],]
)
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)

# quantization algorithm calibration happens here
print('Start Calibration')
os.makedirs('./models', exist_ok=True)


for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train_quant(epoch, trainloader, optimizer, criterion,model_fp32_prepared)
    #test(valloader, model_fp32_prepared)
    #classwise_test(valloader, net)
    test(testloader, model_fp32_prepared)
    # save model checkpoint 
    torch.save(model_fp32_prepared.state_dict(), './models/modelq'+str(epoch)+'.pth')      

print('performing test')
print('Finished Calibration')
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=True)
print(model_int8.conv1.weight())
#res = model_int8(input_fp32)
#checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
#model_int8.load_state_dict(checkpoint)
#model_fp32 = torch.quantization.quantize_dynamic(net, {torch.nn.Linear}, dtype=torch.qint8)
#model_fp32.to(torch.device('cpu'))
#model_fp32.eval()
test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)
os.makedirs('./models', exist_ok=True)
torch.save(model_net8.state_dict(), './models/bestmodel_q.pth')

785121  parameters
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Start Calibration
epoch  1


/opt/conda/lib/python3.7/site-packages/torch/ao/quantization/observer.py:179: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


NameError: name 'train_quant' is not defined

In [ ]:
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=True)
print(model_int8.conv1)
#res = model_int8(input_fp32)
#checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
#model_int8.load_state_dict(checkpoint)
#model_fp32 = torch.quantization.quantize_dynamic(net, {torch.nn.Linear}, dtype=torch.qint8)
#model_fp32.to(torch.device('cpu'))
#model_fp32.eval()
test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)
os.makedirs('./models', exist_ok=True)
torch.save(model_net8.state_dict(), './models/bestmodel_q.pth')

In [ ]:
torch.save(model_fp32_prepared.state_dict(), './models/bestmodel_q2.pth')

In [ ]:
########################################################################
# Train the network
# ^^^^^^^^^^^^^^^^^^^^

def train_quant(epoch, trainloader, optimizer, criterion, net_model):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))
    

In [ ]:
#test QAT
########################################################################
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.quantization as qt

num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
         #QuantStub converts tensors from floating point to quantized
        self.quant = qt.QuantStub()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
       
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)
        self.relu3 = nn.ReLU(inplace=True)
        self.relu4 = nn.ReLU(inplace=True)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc3 = nn.Linear(in_features=256, out_features=33) # 5 is the number of classes here (for batch 3,4,5 out_features is 33)
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = qt.DeQuantStub()
 
    def forward(self, x): 
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.pool(x)
        x = self.conv4(x)
        x = self.relu4(x)
        
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu2(x)
        
        #x = self.pool(x)
        x = self.conv3(x)
        x = self.relu3(x)
        
        x = self.pool(x)
        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fc3(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x 



net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')

model_fp32 = net
model_fp32.train()
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32, [['conv1', 'relu1'],['conv4', 'relu4'],['conv2', 'relu2'],['conv3', 'relu3'],]
)
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)

# quantization algorithm calibration happens here

print('Start Training')
os.makedirs('./models', exist_ok=True)


for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train_quant(epoch, trainloader, optimizer, criterion,model_fp32_prepared)
    #test(valloader, model_fp32_prepared)
    #classwise_test(valloader, net)
    test(testloader, model_fp32_prepared)
    # save model checkpoint 
    torch.save(net.state_dict(), './models/model'+str(epoch)+'.pth')      

print('performing test')
#test(testloader, net)
classwise_test(testloader, net)
print("max accuracy:",np.max(l))
print('Finished Training')
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared)
#res = model_int8(input_fp32)
checkpoint = torch.load('../input/model/bestmodel.pth',map_location=torch.device('cpu'))
net.load_state_dict(checkpoint)

test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)

In [ ]:
########################################################################
# Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models



num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 

class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_1 = torch.quantization.QuantStub()
        self.conv_1_1 = nn.Conv2d(3, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_1_1.weight)
        self.relu_1_2 = nn.ReLU()
        self.norm_1_3 = nn.BatchNorm2d(64)
        self.conv_1_4 = nn.Conv2d(64, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_1_4.weight)
        self.relu_1_5 = nn.ReLU()
        self.norm_1_6 = nn.BatchNorm2d(64)
        self.pool_1_7 = nn.MaxPool2d(2)
        
        self.conv_2_1 = nn.Conv2d(64, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_2_1.weight)        
        self.relu_2_2 = nn.ReLU()
        self.norm_2_3 = nn.BatchNorm2d(128)
        self.conv_2_4 = nn.Conv2d(128, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_2_4.weight)        
        self.relu_2_5 = nn.ReLU()
        self.norm_2_6 = nn.BatchNorm2d(128)
        self.pool_2_7 = nn.MaxPool2d(2)
        
        self.conv_3_1 = nn.Conv2d(128, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_3_1.weight)
        self.relu_3_2 = nn.ReLU()
        self.norm_3_3 = nn.BatchNorm2d(256)
        self.conv_3_4 = nn.Conv2d(256, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_3_4.weight)
        self.relu_3_5 = nn.ReLU()
        self.norm_3_6 = nn.BatchNorm2d(256)
        self.pool_3_7 = nn.MaxPool2d(2)
        
        self.conv_4_1 = nn.Conv2d(256, 512, 3)
        torch.nn.init.kaiming_normal_(self.conv_4_1.weight)
        self.relu_4_2 = nn.ReLU()
        self.norm_4_3 = nn.BatchNorm2d(512)
        self.conv_4_4 = nn.Conv2d(512, 512, 3)
        torch.nn.init.kaiming_normal_(self.conv_4_4.weight)
        self.relu_4_5 = nn.ReLU()
        self.norm_4_6 = nn.BatchNorm2d(512)
        self.dq_1 = torch.quantization.DeQuantStub()
        
        # deconv is the '2D transposed convolution operator'
        self.deconv_5_1 = nn.ConvTranspose2d(512, 256, (2, 2), 2)
        # 61x61 -> 48x48 crop
        self.c_crop_5_2 = lambda x: x[:, :, 6:54, 6:54]
        self.concat_5_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.q_2 = torch.quantization.QuantStub()
        self.conv_5_4 = nn.Conv2d(512, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_5_4.weight)        
        self.relu_5_5 = nn.ReLU()
        self.norm_5_6 = nn.BatchNorm2d(256)
        self.conv_5_7 = nn.Conv2d(256, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_5_7.weight)
        self.relu_5_8 = nn.ReLU()
        self.norm_5_9 = nn.BatchNorm2d(256)
        self.dq_2 = torch.quantization.DeQuantStub()
        
        self.deconv_6_1 = nn.ConvTranspose2d(256, 128, (2, 2), 2)
        # 121x121 -> 88x88 crop
        self.c_crop_6_2 = lambda x: x[:, :, 17:105, 17:105]
        self.concat_6_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.q_3 = torch.quantization.QuantStub()
        self.conv_6_4 = nn.Conv2d(256, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_6_4.weight)
        self.relu_6_5 = nn.ReLU()
        self.norm_6_6 = nn.BatchNorm2d(128)
        self.conv_6_7 = nn.Conv2d(128, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_6_7.weight)
        self.relu_6_8 = nn.ReLU()
        self.norm_6_9 = nn.BatchNorm2d(128)
        self.dq_3 = torch.quantization.DeQuantStub()
        
        self.deconv_7_1 = nn.ConvTranspose2d(128, 64, (2, 2), 2)
        # 252x252 -> 168x168 crop
        self.c_crop_7_2 = lambda x: x[:, :, 44:212, 44:212]
        self.concat_7_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.q_4 = torch.quantization.QuantStub()
        self.conv_7_4 = nn.Conv2d(128, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_7_4.weight)
        self.relu_7_5 = nn.ReLU()
        self.norm_7_6 = nn.BatchNorm2d(64)
        self.conv_7_7 = nn.Conv2d(64, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_7_7.weight)        
        self.relu_7_8 = nn.ReLU()
        self.norm_7_9 = nn.BatchNorm2d(64)
        
        # 1x1 conv ~= fc; n_classes = 33
        self.conv_8_1 = nn.Conv2d(64, 33, 1)
        self.fcout = nn.Linear(in_features=33, out_features=33)
        self.dq_4 = torch.quantization.DeQuantStub()
        
        # residual connections need to be dequantized seperately
        self.dq_resid_1 = torch.quantization.DeQuantStub()
        self.dq_resid_2 = torch.quantization.DeQuantStub()
        self.dq_resid_3 = torch.quantization.DeQuantStub()
        

    def forward(self, x):
        x = self.q_1(x)        
        x = self.conv_1_1(x)
        x = self.relu_1_2(x)
        x = self.norm_1_3(x)
        x = self.conv_1_4(x)
        x = self.relu_1_5(x)
        x_resid_1_quantized = self.norm_1_6(x)
        x = self.pool_1_7(x_resid_1_quantized)
        x_resid_1 = self.dq_resid_1(x_resid_1_quantized)
        
        x = self.conv_2_1(x)
        x = self.relu_2_2(x)
        x = self.norm_2_3(x)
        x = self.conv_2_4(x)
        x = self.relu_2_5(x)
        x_resid_2_quantized = self.norm_2_6(x)
        x = self.pool_2_7(x_resid_2_quantized)
        x_resid_2 = self.dq_resid_2(x_resid_2_quantized)
        
        x = self.conv_3_1(x)
        x = self.relu_3_2(x)
        x = self.norm_3_3(x)
        x = self.conv_3_4(x)
        x = self.relu_3_5(x)
        x_resid_3_quantized = self.norm_3_6(x)
        x = self.pool_3_7(x_resid_3_quantized)
        x_resid_3 = self.dq_resid_3(x_resid_3_quantized)
        
        x = self.conv_4_1(x)
        x = self.relu_4_2(x)
        x = self.norm_4_3(x)        
        x = self.conv_4_4(x)
        x = self.relu_4_5(x)
        x = self.norm_4_6(x)
        x = self.dq_1(x)
        
        x = self.deconv_5_1(x)
        x = self.concat_5_3(self.c_crop_5_2(x_resid_3), x)
        x = self.q_2(x)
        x = self.conv_5_4(x)
        x = self.relu_5_5(x)
        x = self.norm_5_6(x)
        x = self.conv_5_7(x)
        x = self.relu_5_8(x)
        x = self.norm_5_9(x)
        x = self.dq_2(x)
        
        x = self.deconv_6_1(x)
        x = self.concat_6_3(self.c_crop_6_2(x_resid_2), x)
        x = self.q_3(x)
        x = self.conv_6_4(x)
        x = self.relu_6_5(x)
        x = self.norm_6_6(x)
        x = self.conv_6_7(x)
        x = self.relu_6_8(x)
        x = self.norm_6_9(x)
        x = self.dq_3(x)
        
        x = self.deconv_7_1(x)
        x = self.concat_7_3(self.c_crop_7_2(x_resid_1), x)
        x = self.q_4(x)
        x = self.conv_7_4(x)
        x = self.relu_7_5(x)
        x = self.norm_7_6(x)
        x = self.conv_7_7(x)
        x = self.relu_7_8(x)
        x = self.norm_7_9(x)
        
        x = self.conv_8_1(x)
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fcout(x)
        x = self.dq_4(x)        
        return x


In [ ]:
net = UNet()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')
checkpoint = torch.load('../input/models/model18.pth',map_location=torch.device('cpu'))
net.load_state_dict(checkpoint)
net.eval()
model_fp32 = net
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32,
        [
            ['conv_1_1', 'relu_1_2'],
            ['conv_1_4', 'relu_1_5'],
            ['conv_2_1', 'relu_2_2'],
            ['conv_2_4', 'relu_2_5'],
            ['conv_3_1', 'relu_3_2'],
            ['conv_3_4', 'relu_3_5'],
            ['conv_4_1', 'relu_4_2'],
            ['conv_4_4', 'relu_4_5'],
        ]
    )

model_fp32.eval()
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

# quantization algorithm calibration happens here
# this example uses just a single sample, but obvious in prod you will
# want to use some meaningful subset of your training or test set
# instead.
#input_fp32 = torch.randn(64, 3, 3, 3)
#print(input_fp32)
#model_fp32_prepared(input_fp32.cuda())
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared)
#res = model_int8(input_fp32)

#model_fp32 = torch.quantization.quantize_dynamic(net, {torch.nn.Linear}, dtype=torch.qint8)
#model_fp32.to(torch.device('cpu'))
#model_fp32.eval()
test(trainloader,model_int8)
test(testloader, model_int8)
classwise_test(testloader, model_int8)

In [ ]:

########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].

# Apply necessary image transfromations here 

transform = transforms.Compose([ transforms.Resize((164, 164)), transforms.ToTensor(),
                                transforms.Pad(46, padding_mode='reflect'),
                                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])


train_data_dir = '/kaggle/working/4/train' # put path of training dataset
val_data_dir = '/kaggle/working/4/val' # put path of test dataset
test_data_dir = '/kaggle/working/4/test' # put path of test dataset

trainset = torchvision.datasets.ImageFolder(root= train_data_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

valnset = torchvision.datasets.ImageFolder(root= val_data_dir, transform=transform)
valloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root= test_data_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


In [ ]:
########################################################################
# Train the network
# ^^^^^^^^^^^^^^^^^^^^

def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs.size(),labels.size())
        #print(labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))
    

In [ ]:
########################################################################
# Let us look at how the network performs on the test dataset.

l=[]
def test(testloader, model):
    global test_num
    test_num+=1
    correct = 0
    total = 0
    l=[]
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if test_num%2==1:
      print('Accuracy of the network on the valid images: %f %%' % (
                                    100 * correct / total))
    else:
        print('Accuracy of the network on the test images: %f %%' % (
                                    100 * correct / total))
        l.append(100 * correct / total)

#########################################################################
# get details of classes and class to index mapping in a directory
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    classes, _ = find_classes(train_data_dir)
    n_class = len(classes) # number of classes

    class_correct = list(0. for i in range(n_class))
    class_total = list(0. for i in range(n_class))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(n_class):
        print('Accuracy of %10s : %2f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
import torch.nn as nn
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

num_epochs = 20         # desired number of training epochs.
learning_rate = 0.001  
test_num = 0 
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1_1 = nn.Conv2d(3, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_1_1.weight)
        self.relu_1_2 = nn.ReLU()
        self.norm_1_3 = nn.BatchNorm2d(64)
        self.conv_1_4 = nn.Conv2d(64, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_1_4.weight)
        self.relu_1_5 = nn.ReLU()
        self.norm_1_6 = nn.BatchNorm2d(64)
        self.pool_1_7 = nn.MaxPool2d(2)
        
        self.conv_2_1 = nn.Conv2d(64, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_2_1.weight)        
        self.relu_2_2 = nn.ReLU()
        self.norm_2_3 = nn.BatchNorm2d(128)
        self.conv_2_4 = nn.Conv2d(128, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_2_4.weight)        
        self.relu_2_5 = nn.ReLU()
        self.norm_2_6 = nn.BatchNorm2d(128)
        self.pool_2_7 = nn.MaxPool2d(2)
        
        self.conv_3_1 = nn.Conv2d(128, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_3_1.weight)
        self.relu_3_2 = nn.ReLU()
        self.norm_3_3 = nn.BatchNorm2d(256)
        self.conv_3_4 = nn.Conv2d(256, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_3_4.weight)
        self.relu_3_5 = nn.ReLU()
        self.norm_3_6 = nn.BatchNorm2d(256)
        self.pool_3_7 = nn.MaxPool2d(2)
        
        self.conv_4_1 = nn.Conv2d(256, 512, 3)
        torch.nn.init.kaiming_normal_(self.conv_4_1.weight)
        self.relu_4_2 = nn.ReLU()
        self.norm_4_3 = nn.BatchNorm2d(512)
        self.conv_4_4 = nn.Conv2d(512, 512, 3)
        torch.nn.init.kaiming_normal_(self.conv_4_4.weight)
        self.relu_4_5 = nn.ReLU()
        self.norm_4_6 = nn.BatchNorm2d(512)
        
        # deconv is the '2D transposed convolution operator'
        self.deconv_5_1 = nn.ConvTranspose2d(512, 256, (2, 2), 2)
        # 61x61 -> 48x48 crop
        self.c_crop_5_2 = lambda x: x[:, :, 6:54, 6:54]
        self.concat_5_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.conv_5_4 = nn.Conv2d(512, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_5_4.weight)        
        self.relu_5_5 = nn.ReLU()
        self.norm_5_6 = nn.BatchNorm2d(256)
        self.conv_5_7 = nn.Conv2d(256, 256, 3)
        torch.nn.init.kaiming_normal_(self.conv_5_7.weight)
        self.relu_5_8 = nn.ReLU()
        self.norm_5_9 = nn.BatchNorm2d(256)
        
        self.deconv_6_1 = nn.ConvTranspose2d(256, 128, (2, 2), 2)
        # 121x121 -> 88x88 crop
        self.c_crop_6_2 = lambda x: x[:, :, 17:105, 17:105]
        self.concat_6_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.conv_6_4 = nn.Conv2d(256, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_6_4.weight)
        self.relu_6_5 = nn.ReLU()
        self.norm_6_6 = nn.BatchNorm2d(128)
        self.conv_6_7 = nn.Conv2d(128, 128, 3)
        torch.nn.init.kaiming_normal_(self.conv_6_7.weight)
        self.relu_6_8 = nn.ReLU()
        self.norm_6_9 = nn.BatchNorm2d(128)
        
        self.deconv_7_1 = nn.ConvTranspose2d(128, 64, (2, 2), 2)
        # 252x252 -> 168x168 crop
        self.c_crop_7_2 = lambda x: x[:, :, 44:212, 44:212]
        self.concat_7_3 = lambda x, y: torch.cat((x, y), dim=1)
        self.conv_7_4 = nn.Conv2d(128, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_7_4.weight)
        self.relu_7_5 = nn.ReLU()
        self.norm_7_6 = nn.BatchNorm2d(64)
        self.conv_7_7 = nn.Conv2d(64, 64, 3)
        torch.nn.init.kaiming_normal_(self.conv_7_7.weight)        
        self.relu_7_8 = nn.ReLU()
        self.norm_7_9 = nn.BatchNorm2d(64)
        
        # 1x1 conv ~= fc; n_classes = 33
        self.conv_8_1 = nn.Conv2d(64, 33, 1)
        self.fcout = nn.Linear(in_features=33, out_features=33)

    def forward(self, x):
        x = self.conv_1_1(x)
        x = self.relu_1_2(x)
        x = self.norm_1_3(x)
        x = self.conv_1_4(x)
        x = self.relu_1_5(x)
        x_residual_1 = self.norm_1_6(x)
        x = self.pool_1_7(x_residual_1)
        
        x = self.conv_2_1(x)
        x = self.relu_2_2(x)
        x = self.norm_2_3(x)
        x = self.conv_2_4(x)
        x = self.relu_2_5(x)
        x_residual_2 = self.norm_2_6(x)
        x = self.pool_2_7(x_residual_2)
        
        x = self.conv_3_1(x)
        x = self.relu_3_2(x)
        x = self.norm_3_3(x)
        x = self.conv_3_4(x)
        x = self.relu_3_5(x)
        x_residual_3 = self.norm_3_6(x)
        x = self.pool_3_7(x_residual_3)
        
        x = self.conv_4_1(x)
        x = self.relu_4_2(x)
        x = self.norm_4_3(x)        
        x = self.conv_4_4(x)
        x = self.relu_4_5(x)
        x = self.norm_4_6(x)
        
        x = self.deconv_5_1(x)
        x = self.concat_5_3(self.c_crop_5_2(x_residual_3), x)
        x = self.conv_5_4(x)
        x = self.relu_5_5(x)
        x = self.norm_5_6(x)
        x = self.conv_5_7(x)
        x = self.relu_5_8(x)
        x = self.norm_5_9(x)
        
        x = self.deconv_6_1(x)
        x = self.concat_6_3(self.c_crop_6_2(x_residual_2), x)
        x = self.conv_6_4(x)
        x = self.relu_6_5(x)
        x = self.norm_6_6(x)
        x = self.conv_6_7(x)
        x = self.relu_6_8(x)
        x = self.norm_6_9(x)
        
        x = self.deconv_7_1(x)
        x = self.concat_7_3(self.c_crop_7_2(x_residual_1), x)
        x = self.conv_7_4(x)
        x = self.relu_7_5(x)
        x = self.norm_7_6(x)
        x = self.conv_7_7(x)
        x = self.relu_7_8(x)
        x = self.norm_7_9(x)
        
        x = self.conv_8_1(x)
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)
        x = self.fcout(x)
        #x = self.fcout(x)
        
        return x
net = UNet()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()

########################################################################
# Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=3.4e-4)

num_params = np.sum([p.nelement() for p in net.parameters()])
print(num_params, ' parameters')
print('Start Training')
os.makedirs('./models', exist_ok=True)


for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(valloader, net)
    #classwise_test(valloader, net)
    test(testloader, net)
    # save model checkpoint 
    torch.save(net.state_dict(), './models/model'+str(epoch)+'.pth')      

print('performing test')
#test(testloader, net)
classwise_test(testloader, net)
print("max accuracy:",np.max(l))
print('Finished Training')

# Saving our trained model
torch.save(net.state_dict(), './models/bestmodel.pth')
print(net)